In [ ]:
```
Model Risk and Design Memo: Rent Prediction System

Date: January 11, 2026
To: Product & Engineering Teams
From: Data Science
Re: Production Model Design Considerations

---

1. Feature Risks & Exclusions

Features to EXCLUDE even if they improve accuracy:

- Race/ethnicity demographics - Fair Housing Act violations; proxies for protected classes
- School district names - Indirect proxy for neighborhood demographics
- Exact street addresses - Privacy concerns; creates overfitting to specific buildings

Features to include with caution:

- Census education/commuter rates - Legitimate economic indicators but monitor for discriminatory patterns
- Geographic coordinates - Essential for location pricing but round to prevent exact address inference

---

2. Temporal Design

Refresh Frequency: Weekly (Every Monday 3 AM)

Rationale:
- Rental market changes gradually; daily refreshes add noise
- Based on 2022 data showing ~3% quarterly variance
- Balances freshness with model stability

Volatile Market Strategy:
- Exponential time weighting: recent listings weighted 2x vs. 6-month-old data
- Add market velocity feature: 30-day rolling price change by ZIP
- Emergency retraining trigger: week-over-week median prediction error > 10%

---

3. Failure Detection (Silent Degradation)

Monitoring Checks:

1. Data Drift: KS test on feature distributions (p < 0.05 threshold)
2. Prediction Distribution: Alert if 95th percentile prediction > 2x historical
3. Missing Data: Flag if > 20% rows have NULL in key features
4. Geographic Coverage: Error if previously-seen blockgroup has < 5 listings in 90 days
5. Model Staleness: Automated alert if model age > 14 days

---

4. Monotonicity Trade-offs

Expected Monotonic Relationships:
- sqft ↑ → price ↑
- beds ↑ → price ↑ (generally)
- baths ↑ → price ↑

Current Situation: XGBoost has NO monotonicity constraints

- Accuracy gain: ~1-2% better MAPE without constraints
- Risk: Model may predict smaller units cost more (erodes user trust)

Recommendation:

Enforce monotonicity on sqft only (95% correlation with price).

Accept ~0.5% accuracy degradation for explainability.

Beds/baths remain flexible (studio apartments break monotonicity).

Implementation: Use XGBoost's monotone_constraints parameter.

---

Decision Framework: Prioritize user trust and legal compliance over marginal accuracy gains.